In [3]:
# Importation des bibliothèques nécessaires
import pandas as pd
import json
import os
import psycopg2

import boto3
from botocore.exceptions import NoCredentialsError


from psycopg2.extras import Json
from psycopg2.extras import execute_values
import math
from datetime import datetime
from io import BytesIO

In [ ]:
def create_s3_client():
    """
    Creates an S3 client using environment variables for credentials.
    
    :return: Boto3 S3 client
    """
    try:
        # Récupère les identifiants depuis les variables d'environnement
        access_key = os.getenv('SCW_ACCESS_KEY')
        secret_key = os.getenv('SCW_SECRET_KEY')
        endpoint_url = os.getenv('SCW_OBJECT_STORAGE_ENDPOINT')
        region = os.getenv('SCW_REGION')

        if not access_key or not secret_key or not endpoint_url or not region:
            raise ValueError("Identifiants S3 manquants dans les variables d'environnement.")

        # Initialise le client S3
        session = boto3.Session(
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
            region_name=region
        )
        s3_client = session.client('s3', endpoint_url=endpoint_url)

        print("✔️ Client S3 créé avec succès.")
        return s3_client
    except Exception as e:
        print(f"❌ Échec de la création du client S3 : {e}")
        raise


def read_excel_from_s3(s3_client, bucket_name, file_path):
    """
    Retrieves an Excel file from an S3 bucket and loads it as a stream
    
    :param s3_client: S3 client
    :param bucket_name: Name of the S3 bucket
    :param file_path: Key (path) of the file in the S3 bucket
    :return: A file object as a stream
    """
    
    try:
        # Retrieve the file from S3 as a binary stream
        response = s3_client.get_object(Bucket=bucket_name, Key=file_path)
        file_content = response['Body'].read()
        
        # Use BytesIO to create a file-like object from the binary data
        file_stream = BytesIO(file_content)
        
        return file_stream

    except NoCredentialsError:
        print("❌ Les informations d'identification sont manquantes ou incorrectes.")
    except Exception as e:
        print(f"❌ Erreur lors de la récupération du fichier : {e}")
        return None


def load_sheet_from_dataframe(dataframe):
    """
    Converts a Pandas DataFrame to JSON format.
    
    :param dataframe: Pandas DataFrame
    :return: JSON data (list of dictionaries)
    """
    try:
        json_data = dataframe.to_dict(orient='records')
        print(f"✔️ {len(json_data)} lignes converties en JSON.")
        return json_data
    except Exception as e:
        print(f"❌ Erreur lors de la conversion du DataFrame en JSON : {e}")
        raise


def get_connexion():
    """
     Establishes a PostgreSQL connection using environment variables.
    
    :return: PostgreSQL connection object
    """
    try:
        conn = psycopg2.connect(
            dbname=os.getenv('PG_DB_NAME'),
            user=os.getenv('PG_DB_USER'),
            password=os.getenv('PG_DB_PWD'),
            host="localhost",
            port="5432"
        )
        print("✔️ Connexion à PostgreSQL réussie.")
        return conn
    except Exception as e:
        print(f"❌ Échec de la connexion à PostgreSQL : {e}")
        raise


def create_bronze_table(conn, table_name):
    """
    Drops the 'bronze' table if it exists and creates a new one in PostgreSQL.

    :param conn: PostgreSQL connection object
    :param table_name: Name of the table to create
    """
    try:
        cursor = conn.cursor()
        
         # Force drop the table with CASCADE to remove dependencies
        drop_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        cursor.execute(drop_table_query)
        print(f"✔️ Table '{table_name}' supprimée avec CASCADE.")

        # Recreate the table
        create_table_query = f"""
        CREATE TABLE {table_name} (
            id SERIAL PRIMARY KEY,
            data JSONB NOT NULL,
            created_at TIMESTAMPTZ DEFAULT CURRENT_TIMESTAMP
        );
        """
        cursor.execute(drop_table_query)
        cursor.execute(create_table_query)
        conn.commit()
        print(f"✔️ La table '{table_name}' est prête.")
    except Exception as e:
        print(f"❌ Erreur lors de la création de la table '{table_name}' : {e}")
        conn.rollback()
        raise
    finally:
        cursor.close()


def clean_json(obj):
    """
    Cleans JSON data by removing invalid values (e.g., NaN, INF, empty strings).
    
    :param obj: JSON object
    :return: Cleaned JSON object
    """
    if isinstance(obj, dict):
        return {k: clean_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_json(v) for v in obj]
    elif isinstance(obj, float):
        return None if math.isinf(obj) or math.isnan(obj) else obj
    elif isinstance(obj, str):
        return None if obj.upper() in ("INF", "NA", "NAN", "") else obj
    return obj


def import_data_to_bronze_table(conn, table_name, json_data, created_at=None):
    """
    Inserts JSON data into a PostgreSQL table.

    :param conn: PostgreSQL connection object
    :param table_name: Name of the table
    :param json_data: List of dictionaries representing JSON data
    :param created_at: Timestamp for data insertion (defaults to now)
    """
    try:
        cursor = conn.cursor()
        created_at = created_at or datetime.timezone.utc()

        insert_query = f"INSERT INTO {table_name} (created_at, data) VALUES %s"
        json_records = [(created_at, Json(clean_json(record))) for record in json_data]

        execute_values(cursor, insert_query, json_records)
        conn.commit()

        print(f"✔️ {len(json_records)} lignes insérées dans '{table_name}'.")
    except Exception as e:
        print(f"❌ Erreur lors de l'insertion des données dans '{table_name}' : {e}")
        conn.rollback()
        raise
    finally:
        cursor.close()


In [9]:
# Processing time 
created_at = datetime.now()
print(f"\n📅 Début du traitement à {created_at}")

# File name and S3 bucket to process
file_path = "resultats_rpls_2024_v3.xlsx"
bucket_name = os.getenv('SCW_BUCKET_NAME')

# Retrieve the Excel file from S3 as a stream
print(f"\n🚀Chargement du fichier depuis S3 : {file_path} (Bucket: {bucket_name})")
s3_client=create_s3_client()
file_stream = read_excel_from_s3(s3_client, bucket_name, file_path)
print(f"✔️ Fichier chargé")

# Load Excel sheets into pandas DataFrames
print(f"\n🚀Chargement des différentes feuilles du fichier Excel dans des DataFrames pandas")
df_region = pd.read_excel(file_stream, sheet_name="REGION", header=5)
df_departement = pd.read_excel(file_stream, sheet_name="DEPARTEMENT", header=5)
df_commune = pd.read_excel(file_stream, sheet_name="COMMUNES", header=5)
df_epci = pd.read_excel(file_stream, sheet_name="EPCI", header=5)
print(f"✔️ Feuilles chargées")

# Convert DataFrames to JSON for PostgreSQL insertion
print("\n🚀Conversion des DataFrames en JSON")
json_data_region = load_sheet_from_dataframe(df_region)
json_data_departement = load_sheet_from_dataframe(df_departement)
json_data_communes = load_sheet_from_dataframe(df_commune)
json_data_epci = load_sheet_from_dataframe(df_epci)

# Connect to PostgreSQL
print("\n🚀 Connexion à PostgreSQL")
conn = get_connexion()

# Define PostgreSQL table names
bronze_table_name_region = "bronze.logement_rpls_region"
bronze_table_name_departement = "bronze.logement_rpls_departement"
bronze_table_name_commune = "bronze.logement_rpls_commune"
bronze_table_name_epci = "bronze.logement_rpls_epci"

# Create Bronze tables if they don’t exist
print("\n🚀Vérification et création des tables Bronze si nécessaire")
create_bronze_table (conn, bronze_table_name_region)
create_bronze_table (conn, bronze_table_name_departement)
create_bronze_table (conn, bronze_table_name_commune)
create_bronze_table (conn, bronze_table_name_epci)
print("✔️ Tables Bronze prêtes")

# Insert JSON data into Bronze tables
print("\n🚀 Insertion des données dans les tables Bronze")
import_data_to_bronze_table (conn, bronze_table_name_region, json_data_region, created_at)
import_data_to_bronze_table (conn, bronze_table_name_departement, json_data_departement, created_at)
import_data_to_bronze_table (conn, bronze_table_name_commune, json_data_communes, created_at)
import_data_to_bronze_table (conn, bronze_table_name_epci, json_data_epci, created_at)
print("✔️ Données insérées avec succès")

# Close PostgreSQL connection
conn.close()
print("\n✅ Fin du traitement.")



📅 Début du traitement à 2025-04-02 07:21:04.987509

🚀Chargement du fichier depuis S3 : resultats_rpls_2024_v3.xlsx (Bucket: odis-s3)
✔️ Client S3 créé avec succès.
✔️ Fichier chargé

🚀Chargement des différentes feuilles du fichier Excel dans des DataFrames pandas
✔️ Feuilles chargées

🚀Conversion des DataFrames en JSON
✔️ 23 lignes converties en JSON.
✔️ 96 lignes converties en JSON.
✔️ 16859 lignes converties en JSON.
✔️ 1325 lignes converties en JSON.

🚀 Connexion à PostgreSQL
✔️ Connexion à PostgreSQL réussie.

🚀Vérification et création des tables Bronze si nécessaire
❌ Erreur lors de la création de la table 'bronze.bronze.logement_rpls_region' : cross-database references are not implemented: "bronze.bronze.logement_rpls_region"



FeatureNotSupported: cross-database references are not implemented: "bronze.bronze.logement_rpls_region"
